<< [Sudoku-8-SimulatedAnnealing](Sudoku-8-SimulatedAnnealing.ipynb) | [Index](README.md) | [Sudoku-10-NeuralNetwork](Sudoku-10-NeuralNetwork.ipynb) >>

# Résolution de Sudoku par Stratégies Humaines

## Objectifs d'apprentissage

A la fin de ce notebook, vous saurez :
1. Comprendre comment les experts humains resolvent les Sudoku, par opposition aux algorithmes de force brute
2. Implementer les techniques de base : Naked Singles et Hidden Singles
3. Implementer les techniques intermediaires : Naked Pairs, Locked Candidates
4. Implementer la technique avancee X-Wing
5. Combiner ces techniques dans un solveur hybride avec fallback vers le backtracking
6. Analyser quelles techniques sont necessaires selon la difficulte du puzzle

### Prerequis
- [Sudoku-0-Environment](Sudoku-0-Environment.ipynb) : classes `SudokuGrid`, `ISudokuSolver`, `SudokuHelper`
- Familiarite avec les regles du Sudoku (lignes, colonnes, blocs 3x3)

### Duree estimee : 60 minutes

## Importation de l'environnement

Nous importons les classes de base definies dans le notebook `Sudoku-0-Environment` : `SudokuGrid`, `ISudokuSolver`, `SudokuHelper`.

In [ ]:
#!import Sudoku-0-Environment.ipynb

## 1. Introduction : Comment les humains resolvent les Sudoku (~3 min)

Les algorithmes que nous avons vus dans les notebooks precedents (backtracking, OR-Tools, Dancing Links, etc.) resolvent les Sudoku par **force brute** ou par **satisfaction de contraintes** de maniere systematique. Un expert humain procede tout autrement : il applique des **techniques de deduction logique** de difficulte croissante, en commencant par les plus simples.

### Hierarchie des techniques

Le projet [Sudoku.Human](https://github.com/jsboigeEpita/2024-EPITA-SCIA-PPC-Sudoku-NLP) identifie 13 techniques, classees par difficulte :

| Niveau | Technique | Principe |
|--------|-----------|----------|
| **Basique** | Naked Singles | Une cellule n'a qu'un seul candidat possible |
| **Basique** | Hidden Singles | Une valeur ne peut aller qu'a un seul endroit dans une unite |
| **Intermediaire** | Naked Pairs/Triples | N cellules partagent les memes N candidats dans une unite |
| **Intermediaire** | Hidden Pairs/Triples | N valeurs sont restreintes aux memes N cellules dans une unite |
| **Intermediaire** | Locked Candidates (Pointing) | Candidats dans un bloc restreints a une seule ligne/colonne |
| **Intermediaire** | Locked Candidates (Claiming) | Candidats dans une ligne/colonne restreints a un seul bloc |
| **Avance** | X-Wing | 2 lignes ou le meme candidat est restreint aux memes 2 colonnes |
| **Avance** | Y-Wing | Chaine de 3 cellules avec pattern de candidats specifique |
| **Avance** | XYZ-Wing | Extension du Y-Wing a 3 candidats |
| **Expert** | Swordfish | Extension de X-Wing a 3 lignes/colonnes |
| **Expert** | Jellyfish | Extension de X-Wing a 4 lignes/colonnes |
| **Expert** | 3D Medusa | Technique de coloriage sur graphe de candidats |
| **Expert** | Unique Rectangles | Evitement de patterns mortels (deadly patterns) |

### Approche du notebook

Nous allons implementer les techniques les plus courantes et les combiner dans un solveur `HumanSolver` qui :
1. Maintient une grille de **candidats** pour chaque cellule vide
2. Applique les techniques dans l'ordre de difficulte croissante
3. Recommence au debut a chaque progres
4. Utilise le **backtracking** en dernier recours si aucune technique ne fonctionne

## 2. Infrastructure : Grille de candidats

Avant d'implementer les techniques, nous avons besoin d'une structure de donnees pour gerer les **candidats** de chaque cellule. Pour chaque cellule vide, nous maintenons l'ensemble des valeurs encore possibles.

La classe `CandidateGrid` encapsule cette logique et fournit des methodes utilitaires pour :
- Initialiser les candidats a partir d'une grille de Sudoku
- Eliminer un candidat d'une cellule (avec propagation aux voisins)
- Placer une valeur dans une cellule
- Interroger les candidats d'une unite (ligne, colonne, bloc)

In [ ]:
/// <summary>
/// Grille de candidats pour le solveur humain.
/// Chaque cellule vide possede un ensemble de valeurs candidates (1-9).
/// </summary>
public class CandidateGrid
{
    // Grille 9x9 de HashSet : null si la cellule est resolue, sinon les candidats
    public HashSet<int>[,] Candidates { get; private set; } = new HashSet<int>[9, 9];
    
    // Reference vers la grille de Sudoku associee
    public SudokuGrid Grid { get; private set; }
    
    /// <summary>
    /// Initialise les candidats a partir d'une grille de Sudoku.
    /// Pour chaque cellule vide, les candidats sont les valeurs non presentes chez les voisins.
    /// </summary>
    public CandidateGrid(SudokuGrid grid)
    {
        Grid = grid;
        for (int row = 0; row < 9; row++)
        {
            for (int col = 0; col < 9; col++)
            {
                if (grid.Cells[row, col] == 0)
                {
                    Candidates[row, col] = new HashSet<int>(grid.GetAvailableNumbers(row, col));
                }
                else
                {
                    Candidates[row, col] = null; // Cellule deja resolue
                }
            }
        }
    }
    
    /// <summary>
    /// Place une valeur dans une cellule et elimine cette valeur des candidats des voisins.
    /// Retourne true si le placement est valide (pas de conflit).
    /// </summary>
    public bool PlaceValue(int row, int col, int value)
    {
        Grid.Cells[row, col] = value;
        Candidates[row, col] = null;
        
        // Eliminer la valeur des candidats de tous les voisins
        foreach (var (nRow, nCol) in SudokuGrid.CellNeighbours[row][col])
        {
            if (Candidates[nRow, nCol] != null)
            {
                Candidates[nRow, nCol].Remove(value);
                if (Candidates[nRow, nCol].Count == 0)
                    return false; // Conflit : un voisin n'a plus de candidats
            }
        }
        return true;
    }
    
    /// <summary>
    /// Elimine un candidat d'une cellule. Retourne true si le candidat etait present.
    /// </summary>
    public bool EliminateCandidate(int row, int col, int value)
    {
        if (Candidates[row, col] == null) return false;
        return Candidates[row, col].Remove(value);
    }
    
    /// <summary>
    /// Retourne les cellules non resolues d'une unite donnee (ligne, colonne ou bloc).
    /// </summary>
    public List<(int row, int col, HashSet<int> candidates)> GetUnitCells((int row, int col)[] unit)
    {
        var result = new List<(int, int, HashSet<int>)>();
        foreach (var (r, c) in unit)
        {
            if (Candidates[r, c] != null && Candidates[r, c].Count > 0)
            {
                result.Add((r, c, Candidates[r, c]));
            }
        }
        return result;
    }
    
    /// <summary>
    /// Verifie si le puzzle est completement resolu.
    /// </summary>
    public bool IsSolved()
    {
        for (int row = 0; row < 9; row++)
            for (int col = 0; col < 9; col++)
                if (Grid.Cells[row, col] == 0) return false;
        return true;
    }
    
    /// <summary>
    /// Nombre de cellules encore non resolues.
    /// </summary>
    public int RemainingCells()
    {
        int count = 0;
        for (int row = 0; row < 9; row++)
            for (int col = 0; col < 9; col++)
                if (Grid.Cells[row, col] == 0) count++;
        return count;
    }
    
    /// <summary>
    /// Affiche un resume de l'etat des candidats.
    /// </summary>
    public string Summary()
    {
        int totalCells = 0, totalCandidates = 0;
        for (int row = 0; row < 9; row++)
            for (int col = 0; col < 9; col++)
                if (Candidates[row, col] != null)
                {
                    totalCells++;
                    totalCandidates += Candidates[row, col].Count;
                }
        return $"Cellules non resolues: {totalCells}, Candidats totaux: {totalCandidates}, " +
               $"Moyenne: {(totalCells > 0 ? (double)totalCandidates / totalCells : 0):F1} candidats/cellule";
    }
}

### Interpretation : CandidateGrid

La classe `CandidateGrid` est le fondement de toute l'approche humaine. Contrairement au backtracking qui teste les valeurs une a une, ici on maintient en permanence la liste des **valeurs possibles** pour chaque cellule. C'est exactement ce que fait un joueur humain quand il note les petits chiffres au crayon dans les cases.

| Methode | Role |
|---------|------|
| `PlaceValue` | Place une valeur et propage l'elimination aux voisins |
| `EliminateCandidate` | Retire un candidat d'une cellule specifique |
| `GetUnitCells` | Recupere les cellules non resolues d'une unite (ligne/colonne/bloc) |
| `IsSolved` | Verifie si le puzzle est completement resolu |

Testons cette infrastructure sur un puzzle facile.

In [ ]:
// Test de la grille de candidats sur un puzzle facile
var testGrid = SudokuHelper.GetSudokus(SudokuDifficulty.Easy).First();
display($"Puzzle initial :\n{testGrid}");

var candidates = new CandidateGrid(testGrid);
display(candidates.Summary());

// Afficher les candidats de quelques cellules
for (int row = 0; row < 3; row++)
{
    for (int col = 0; col < 3; col++)
    {
        if (candidates.Candidates[row, col] != null)
        {
            var vals = string.Join(", ", candidates.Candidates[row, col].OrderBy(v => v));
            display($"  Cellule ({row},{col}) : candidats = {{{vals}}}");
        }
    }
}

### Interpretation : Grille de candidats

La sortie montre le fonctionnement de la classe `CandidateGrid` sur un puzzle facile :

| Metrique | Valeur attendue | Signification |
|----------|----------------|---------------|
| Cellules non resolues | ~40-50 | Nombre de cases vides au depart |
| Candidats totaux | ~200-300 | Somme des candidats pour toutes les cellules vides |
| Moyenne candidats/cellule | ~4-6 | Nombre moyen de possibilites par case |

**Observation** : Les cellules dans le coin superieur gauche (0,0), (0,1), (0,2), etc. affichent leurs candidats respectifs. Cette grille de candidats est la base de toutes les techniques humaines qui suivent.

> **Note technique** : La methode `Summary()` fournit une vue macroscopique de l'etat du puzzle, utile pour suivre la progression de la resolution.

## 3. Techniques de base (~5 min)

Les techniques de base suffisent pour resoudre la plupart des puzzles classes "Facile". Elles reposent sur un principe simple : quand il ne reste qu'une seule possibilite, on peut placer la valeur avec certitude.

### 3.1 Naked Singles (Singleton nu)

**Principe** : Si une cellule n'a qu'un seul candidat, ce candidat est forcement la valeur de cette cellule.

C'est la technique la plus simple et la plus intuitive. Quand on elimine suffisamment de candidats par les contraintes de ligne, colonne et bloc, il ne reste parfois qu'un seul choix possible.

$$\text{Si } |\text{Candidats}(r, c)| = 1 \Rightarrow \text{Placer}(r, c, \text{unique candidat})$$

### 3.2 Hidden Singles (Singleton cache)

**Principe** : Si une valeur n'apparait comme candidat que dans **une seule cellule** d'une unite (ligne, colonne ou bloc), alors cette cellule doit contenir cette valeur, meme si elle a d'autres candidats.

$$\forall \text{unite } U, \forall v \in \{1..9\} : |\{c \in U : v \in \text{Candidats}(c)\}| = 1 \Rightarrow \text{Placer}(c, v)$$

Cette technique est plus puissante que les Naked Singles car elle peut trouver des placements meme quand une cellule a plusieurs candidats.

In [ ]:
/// <summary>
/// Contient les techniques de resolution humaines.
/// Chaque technique retourne le nombre de placements ou eliminations effectues.
/// </summary>
public static class HumanTechniques
{
    /// <summary>
    /// Naked Singles : place les cellules qui n'ont qu'un seul candidat.
    /// Retourne le nombre de placements effectues.
    /// </summary>
    public static int ApplyNakedSingles(CandidateGrid cg)
    {
        int placements = 0;
        bool progress = true;
        
        while (progress)
        {
            progress = false;
            for (int row = 0; row < 9; row++)
            {
                for (int col = 0; col < 9; col++)
                {
                    if (cg.Candidates[row, col] != null && cg.Candidates[row, col].Count == 1)
                    {
                        int value = cg.Candidates[row, col].First();
                        cg.PlaceValue(row, col, value);
                        placements++;
                        progress = true;
                    }
                }
            }
        }
        return placements;
    }
    
    /// <summary>
    /// Hidden Singles : pour chaque unite, si une valeur n'a qu'un seul emplacement possible,
    /// la placer dans cette cellule.
    /// Retourne le nombre de placements effectues.
    /// </summary>
    public static int ApplyHiddenSingles(CandidateGrid cg)
    {
        int placements = 0;
        
        // Parcourir toutes les unites (9 lignes + 9 colonnes + 9 blocs = 27 unites)
        foreach (var unit in SudokuGrid.AllNeighbours)
        {
            var cells = cg.GetUnitCells(unit);
            if (cells.Count == 0) continue;
            
            // Pour chaque valeur de 1 a 9
            for (int val = 1; val <= 9; val++)
            {
                // Trouver les cellules de cette unite qui ont val comme candidat
                var cellsWithVal = cells.Where(c => c.candidates.Contains(val)).ToList();
                
                if (cellsWithVal.Count == 1)
                {
                    var (row, col, _) = cellsWithVal[0];
                    if (cg.Grid.Cells[row, col] == 0) // Pas encore place
                    {
                        cg.PlaceValue(row, col, val);
                        placements++;
                    }
                }
            }
        }
        return placements;
    }
}

### Test des techniques de base

Testons ces deux techniques sur un puzzle facile. Les puzzles faciles sont generalement resolubles uniquement avec Naked Singles et Hidden Singles.

In [ ]:
// Test sur un puzzle facile
var easyPuzzle = (SudokuGrid)SudokuHelper.GetSudokus(SudokuDifficulty.Easy).First().Clone();
var cg = new CandidateGrid(easyPuzzle);

display($"Puzzle facile initial : {cg.RemainingCells()} cellules vides");
display(cg.Summary());

int totalPlacements = 0;
int round = 0;
bool madeProgress = true;

while (madeProgress && !cg.IsSolved())
{
    madeProgress = false;
    round++;
    
    int ns = HumanTechniques.ApplyNakedSingles(cg);
    if (ns > 0)
    {
        display($"  Tour {round} - Naked Singles : {ns} placements");
        totalPlacements += ns;
        madeProgress = true;
    }
    
    int hs = HumanTechniques.ApplyHiddenSingles(cg);
    if (hs > 0)
    {
        display($"  Tour {round} - Hidden Singles : {hs} placements");
        totalPlacements += hs;
        madeProgress = true;
    }
}

display($"\nResultat : {(cg.IsSolved() ? "RESOLU" : "BLOQUE")} apres {totalPlacements} placements en {round} tours");
display($"{easyPuzzle}");

### Interpretation : Techniques de base

**Observation** : Les techniques de base (Naked Singles + Hidden Singles) suffisent generalement pour resoudre les puzzles de difficulte "Easy". C'est conforme a la classification standard des puzzles de Sudoku.

| Technique | Quand l'utiliser | Complexite cognitive |
|-----------|-----------------|---------------------|
| Naked Singles | Chaque iteration, en premier | Tres faible |
| Hidden Singles | Apres les Naked Singles | Faible |

> **Note** : Les Hidden Singles sont souvent plus productifs que les Naked Singles car ils exploitent la contrainte d'unicite au sein d'une unite, meme quand une cellule a plusieurs candidats.

Essayons maintenant sur un puzzle de difficulte moyenne pour voir les limites de ces techniques.

In [ ]:
// Test sur un puzzle moyen : les techniques de base ne suffisent plus
var mediumPuzzle = (SudokuGrid)SudokuHelper.GetSudokus(SudokuDifficulty.Medium).First().Clone();
var cgMedium = new CandidateGrid(mediumPuzzle);

display($"Puzzle moyen initial : {cgMedium.RemainingCells()} cellules vides");

int totalMedium = 0;
bool progress = true;
while (progress && !cgMedium.IsSolved())
{
    progress = false;
    int ns = HumanTechniques.ApplyNakedSingles(cgMedium);
    int hs = HumanTechniques.ApplyHiddenSingles(cgMedium);
    totalMedium += ns + hs;
    progress = (ns + hs) > 0;
}

display($"Resultat avec techniques de base uniquement : {(cgMedium.IsSolved() ? "RESOLU" : "BLOQUE")}");
display($"Placements effectues : {totalMedium}, Cellules restantes : {cgMedium.RemainingCells()}");
display(cgMedium.Summary());

### Interpretation : Limites des techniques de base

Comme on le voit, les techniques de base se bloquent sur les puzzles de difficulte moyenne. Il reste des cellules non resolues car aucune cellule n'a un seul candidat et aucune valeur n'est restreinte a une seule position dans ses unites. Il faut des techniques plus sophistiquees pour progresser.

C'est la que les techniques intermediaires entrent en jeu : elles ne placent pas directement de valeurs, mais **eliminent des candidats**, ce qui peut debloquer les Naked Singles et Hidden Singles.

## 4. Techniques intermediaires (~5 min)

Les techniques intermediaires operent par **elimination de candidats** plutot que par placement direct de valeurs. Elles identifient des patterns dans les candidats qui permettent de deduire que certaines valeurs sont impossibles dans certaines cellules.

### 4.1 Naked Pairs (Paires nues)

**Principe** : Si deux cellules dans une meme unite contiennent exactement les memes deux candidats, alors ces deux valeurs sont "reservees" pour ces deux cellules. On peut les eliminer des autres cellules de l'unite.

**Exemple** : Si dans une ligne, les cellules A et B ont toutes deux les candidats {3, 7}, alors 3 et 7 seront forcement dans A et B. On peut eliminer 3 et 7 des candidats de toutes les autres cellules de cette ligne.

$$\text{Si } C_1 = C_2 = \{a, b\} \text{ dans unite } U \Rightarrow \forall c \in U \setminus \{C_1, C_2\} : \text{eliminer } a, b \text{ de } C_c$$

Ce principe se generalise aux **Naked Triples** (3 cellules, 3 candidats) et au-dela.

### 4.2 Locked Candidates (Candidats verrouilles)

Il existe deux variantes :

**Pointing** : Si dans un bloc 3x3, un candidat n'apparait que dans une seule ligne (ou colonne), alors ce candidat peut etre elimine des autres cellules de cette ligne (ou colonne) en dehors du bloc.

**Claiming** : Si dans une ligne (ou colonne), un candidat n'apparait que dans un seul bloc, alors ce candidat peut etre elimine des autres cellules de ce bloc en dehors de la ligne (ou colonne).

In [ ]:
// Ajout des techniques intermediaires a la classe HumanTechniques
public static class HumanTechniquesIntermediate
{
    /// <summary>
    /// Naked Pairs : dans chaque unite, si 2 cellules ont exactement les memes 2 candidats,
    /// eliminer ces candidats des autres cellules de l'unite.
    /// Retourne le nombre d'eliminations effectuees.
    /// </summary>
    public static int ApplyNakedPairs(CandidateGrid cg)
    {
        int eliminations = 0;
        
        foreach (var unit in SudokuGrid.AllNeighbours)
        {
            var cells = cg.GetUnitCells(unit);
            
            // Chercher les cellules avec exactement 2 candidats
            var pairCells = cells.Where(c => c.candidates.Count == 2).ToList();
            
            for (int i = 0; i < pairCells.Count; i++)
            {
                for (int j = i + 1; j < pairCells.Count; j++)
                {
                    // Verifier si les deux cellules ont les memes candidats
                    if (pairCells[i].candidates.SetEquals(pairCells[j].candidates))
                    {
                        var pairValues = pairCells[i].candidates;
                        
                        // Eliminer ces valeurs des autres cellules de l'unite
                        foreach (var cell in cells)
                        {
                            if ((cell.row, cell.col) != (pairCells[i].row, pairCells[i].col) &&
                                (cell.row, cell.col) != (pairCells[j].row, pairCells[j].col))
                            {
                                foreach (int val in pairValues)
                                {
                                    if (cg.EliminateCandidate(cell.row, cell.col, val))
                                        eliminations++;
                                }
                            }
                        }
                    }
                }
            }
        }
        return eliminations;
    }
    
    /// <summary>
    /// Locked Candidates (Pointing) : si dans un bloc, un candidat n'est present
    /// que dans une seule ligne ou colonne, l'eliminer du reste de cette ligne/colonne.
    /// Retourne le nombre d'eliminations effectuees.
    /// </summary>
    public static int ApplyLockedCandidatesPointing(CandidateGrid cg)
    {
        int eliminations = 0;
        
        // Parcourir les 9 blocs 3x3
        for (int box = 0; box < 9; box++)
        {
            int startRow = (box / 3) * 3;
            int startCol = (box % 3) * 3;
            
            for (int val = 1; val <= 9; val++)
            {
                // Trouver les positions du candidat val dans ce bloc
                var positions = new List<(int row, int col)>();
                for (int r = startRow; r < startRow + 3; r++)
                    for (int c = startCol; c < startCol + 3; c++)
                        if (cg.Candidates[r, c] != null && cg.Candidates[r, c].Contains(val))
                            positions.Add((r, c));
                
                if (positions.Count < 2) continue;
                
                // Verifier si toutes les positions sont sur la meme ligne
                if (positions.All(p => p.row == positions[0].row))
                {
                    int row = positions[0].row;
                    // Eliminer val du reste de cette ligne (hors du bloc)
                    for (int c = 0; c < 9; c++)
                    {
                        if (c < startCol || c >= startCol + 3) // Hors du bloc
                        {
                            if (cg.EliminateCandidate(row, c, val))
                                eliminations++;
                        }
                    }
                }
                
                // Verifier si toutes les positions sont sur la meme colonne
                if (positions.All(p => p.col == positions[0].col))
                {
                    int col = positions[0].col;
                    // Eliminer val du reste de cette colonne (hors du bloc)
                    for (int r = 0; r < 9; r++)
                    {
                        if (r < startRow || r >= startRow + 3) // Hors du bloc
                        {
                            if (cg.EliminateCandidate(r, col, val))
                                eliminations++;
                        }
                    }
                }
            }
        }
        return eliminations;
    }
    
    /// <summary>
    /// Locked Candidates (Claiming) : si dans une ligne/colonne, un candidat n'est present
    /// que dans un seul bloc, l'eliminer du reste de ce bloc.
    /// Retourne le nombre d'eliminations effectuees.
    /// </summary>
    public static int ApplyLockedCandidatesClaiming(CandidateGrid cg)
    {
        int eliminations = 0;
        
        for (int val = 1; val <= 9; val++)
        {
            // Verifier chaque ligne
            for (int row = 0; row < 9; row++)
            {
                var cols = new List<int>();
                for (int c = 0; c < 9; c++)
                    if (cg.Candidates[row, c] != null && cg.Candidates[row, c].Contains(val))
                        cols.Add(c);
                
                if (cols.Count < 2) continue;
                
                // Verifier si toutes les colonnes sont dans le meme bloc
                int boxCol = cols[0] / 3;
                if (cols.All(c => c / 3 == boxCol))
                {
                    int startCol = boxCol * 3;
                    int startRow = (row / 3) * 3;
                    // Eliminer val du reste du bloc (hors de cette ligne)
                    for (int r = startRow; r < startRow + 3; r++)
                    {
                        if (r != row)
                        {
                            for (int c = startCol; c < startCol + 3; c++)
                            {
                                if (cg.EliminateCandidate(r, c, val))
                                    eliminations++;
                            }
                        }
                    }
                }
            }
            
            // Verifier chaque colonne
            for (int col = 0; col < 9; col++)
            {
                var rows = new List<int>();
                for (int r = 0; r < 9; r++)
                    if (cg.Candidates[r, col] != null && cg.Candidates[r, col].Contains(val))
                        rows.Add(r);
                
                if (rows.Count < 2) continue;
                
                // Verifier si toutes les lignes sont dans le meme bloc
                int boxRow = rows[0] / 3;
                if (rows.All(r => r / 3 == boxRow))
                {
                    int startRow = boxRow * 3;
                    int startCol = (col / 3) * 3;
                    // Eliminer val du reste du bloc (hors de cette colonne)
                    for (int r = startRow; r < startRow + 3; r++)
                    {
                        for (int c = startCol; c < startCol + 3; c++)
                        {
                            if (c != col)
                            {
                                if (cg.EliminateCandidate(r, c, val))
                                    eliminations++;
                            }
                        }
                    }
                }
            }
        }
        return eliminations;
    }
}

### Interpretation : Techniques intermediaires

Les techniques intermediaires fonctionnent differemment des techniques de base :

| Aspect | Techniques de base | Techniques intermediaires |
|--------|-------------------|-------------------------|
| Action | **Placent** des valeurs | **Eliminent** des candidats |
| Effet | Reduction directe des cellules vides | Reduction indirecte (deblocage pour les techniques de base) |
| Complexite | O(81) par passe | O(81 * 9) par passe |

L'enchainement typique est :
1. Appliquer les eliminations (Naked Pairs, Locked Candidates)
2. Reappliquer les techniques de base (Naked Singles, Hidden Singles)
3. Si progres, recommencer au point 1

> **Note** : Les **Hidden Pairs** (N valeurs restreintes aux memes N cellules dans une unite, eliminer les autres candidats de ces cellules) sont le dual des Naked Pairs mais ne sont pas implementes ici par souci de concision.

## 5. Technique avancee : X-Wing (~5 min)

### Principe du X-Wing

Le X-Wing est une technique basee sur les **lignes et colonnes**. Le pattern est le suivant :

Si un candidat `v` n'apparait que dans **exactement 2 colonnes** dans **2 lignes differentes**, et que ces 2 colonnes sont les **memes** dans les 2 lignes, alors `v` peut etre elimine de toutes les autres cellules de ces 2 colonnes.

```
Ligne r1 : v possible en (r1, c1) et (r1, c2) uniquement
Ligne r2 : v possible en (r2, c1) et (r2, c2) uniquement
           => v est forcement dans les coins du rectangle
           => Eliminer v de (c1, autres lignes) et (c2, autres lignes)
```

Le meme raisonnement s'applique en inversant lignes et colonnes.

**Generalisation** : Le Swordfish (3 lignes, 3 colonnes) et le Jellyfish (4 lignes, 4 colonnes) suivent le meme principe avec plus de lignes/colonnes.

In [ ]:
public static class HumanTechniquesAdvanced
{
    /// <summary>
    /// X-Wing : si un candidat v apparait dans exactement 2 colonnes pour 2 lignes,
    /// eliminer v des autres cellules de ces colonnes.
    /// Applique aussi la variante colonne (2 lignes dans 2 colonnes).
    /// Retourne le nombre d'eliminations effectuees.
    /// </summary>
    public static int ApplyXWing(CandidateGrid cg)
    {
        int eliminations = 0;
        
        for (int val = 1; val <= 9; val++)
        {
            // X-Wing sur les lignes
            eliminations += ApplyXWingOnRows(cg, val);
            
            // X-Wing sur les colonnes
            eliminations += ApplyXWingOnCols(cg, val);
        }
        
        return eliminations;
    }
    
    private static int ApplyXWingOnRows(CandidateGrid cg, int val)
    {
        int eliminations = 0;
        
        // Pour chaque ligne, trouver les colonnes ou val est candidat
        var rowPositions = new List<int>[9];
        for (int r = 0; r < 9; r++)
        {
            rowPositions[r] = new List<int>();
            for (int c = 0; c < 9; c++)
                if (cg.Candidates[r, c] != null && cg.Candidates[r, c].Contains(val))
                    rowPositions[r].Add(c);
        }
        
        // Chercher 2 lignes avec exactement les memes 2 colonnes
        for (int r1 = 0; r1 < 9; r1++)
        {
            if (rowPositions[r1].Count != 2) continue;
            
            for (int r2 = r1 + 1; r2 < 9; r2++)
            {
                if (rowPositions[r2].Count != 2) continue;
                
                if (rowPositions[r1][0] == rowPositions[r2][0] &&
                    rowPositions[r1][1] == rowPositions[r2][1])
                {
                    int c1 = rowPositions[r1][0];
                    int c2 = rowPositions[r1][1];
                    
                    // Eliminer val des colonnes c1 et c2 (hors lignes r1, r2)
                    for (int r = 0; r < 9; r++)
                    {
                        if (r != r1 && r != r2)
                        {
                            if (cg.EliminateCandidate(r, c1, val)) eliminations++;
                            if (cg.EliminateCandidate(r, c2, val)) eliminations++;
                        }
                    }
                }
            }
        }
        return eliminations;
    }
    
    private static int ApplyXWingOnCols(CandidateGrid cg, int val)
    {
        int eliminations = 0;
        
        // Pour chaque colonne, trouver les lignes ou val est candidat
        var colPositions = new List<int>[9];
        for (int c = 0; c < 9; c++)
        {
            colPositions[c] = new List<int>();
            for (int r = 0; r < 9; r++)
                if (cg.Candidates[r, c] != null && cg.Candidates[r, c].Contains(val))
                    colPositions[c].Add(r);
        }
        
        // Chercher 2 colonnes avec exactement les memes 2 lignes
        for (int c1 = 0; c1 < 9; c1++)
        {
            if (colPositions[c1].Count != 2) continue;
            
            for (int c2 = c1 + 1; c2 < 9; c2++)
            {
                if (colPositions[c2].Count != 2) continue;
                
                if (colPositions[c1][0] == colPositions[c2][0] &&
                    colPositions[c1][1] == colPositions[c2][1])
                {
                    int r1 = colPositions[c1][0];
                    int r2 = colPositions[c1][1];
                    
                    // Eliminer val des lignes r1 et r2 (hors colonnes c1, c2)
                    for (int c = 0; c < 9; c++)
                    {
                        if (c != c1 && c != c2)
                        {
                            if (cg.EliminateCandidate(r1, c, val)) eliminations++;
                            if (cg.EliminateCandidate(r2, c, val)) eliminations++;
                        }
                    }
                }
            }
        }
        return eliminations;
    }
}

### Interpretation : X-Wing

Le X-Wing est la premiere technique qui raisonne sur **plusieurs unites simultanement**. Les techniques precedentes (Naked Singles, Hidden Singles, Naked Pairs, Locked Candidates) raisonnent unite par unite.

| Technique | Portee | Type de pattern |
|-----------|--------|----------------|
| Naked/Hidden Singles | 1 unite | Unicite |
| Naked Pairs | 1 unite | Exclusion mutuelle |
| Locked Candidates | 2 unites (bloc + ligne/col) | Intersection |
| X-Wing | 2 lignes + 2 colonnes | Rectangle |

> **Pour aller plus loin** : Les techniques Y-Wing et XYZ-Wing forment des chaines de cellules bivalue. Le Y-Wing utilise 3 cellules avec 2 candidats chacune, formant un "pivot" et deux "pinces". Toute cellule visible par les deux pinces peut voir ses candidats reduits. Ces techniques sont trop complexes pour etre detaillees ici mais suivent le meme principe d'elimination par deduction logique.

## 6. Techniques expertes (apercu) (~3 min)

Les techniques suivantes sont mentionnees pour reference. Elles sont implementees dans le projet [Sudoku.Human](https://github.com/jsboigeEpita/2024-EPITA-SCIA-PPC-Sudoku-NLP) mais leur complexite depasse le cadre de ce notebook.

### 6.1 Fish Patterns (Swordfish, Jellyfish)

Generalisation du X-Wing a N lignes/colonnes :

| Pattern | Dimension | Description |
|---------|-----------|-------------|
| X-Wing | 2x2 | 2 lignes, 2 colonnes |
| Swordfish | 3x3 | 3 lignes, 3 colonnes (candidat dans au plus 3 colonnes par ligne) |
| Jellyfish | 4x4 | 4 lignes, 4 colonnes |

Le principe est identique : si un candidat est restreint a au plus N colonnes dans N lignes, et que l'ensemble des colonnes est le meme, on peut eliminer ce candidat des autres cellules de ces colonnes.

### 6.2 3D Medusa (Coloriage)

La 3D Medusa est une technique de **coloriage** (coloring). Elle construit un graphe ou les noeuds sont les candidats et les aretes representent les relations "si ce candidat est vrai, alors cet autre est faux" (conjugues). En alternant deux couleurs, on peut deduire des eliminations par contradiction.

### 6.3 Rectangles (Unique, Hidden, Avoidable)

Ces techniques exploitent le fait qu'un Sudoku bien pose a une **solution unique**. Si un pattern de candidats cree un "deadly pattern" (rectangle mortel) qui admettrait deux solutions, on peut eliminer les candidats qui y participent.

**Unique Rectangle** : 4 cellules aux coins d'un rectangle (dans 2 lignes, 2 colonnes, 2 blocs) avec les memes 2 candidats. Si ce pattern existe, il faut le casser pour preserver l'unicite de la solution.

## 7. Solveur humain complet (~4 min)

Nous assemblons maintenant toutes les techniques dans un solveur `HumanSolver` qui implemente `ISudokuSolver`. Le solveur :

1. **Initialise** la grille de candidats
2. **Boucle** : applique les techniques dans l'ordre de difficulte croissante
3. **Trace** quelle technique a ete utilisee a chaque etape
4. **Fallback** : si aucune technique humaine ne progresse, utilise le backtracking

Cette approche hybride garantit la resolution de tout puzzle valide, tout en privilegiant les techniques humaines tant que possible.

In [ ]:
/// <summary>
/// Solveur de Sudoku imitant le raisonnement humain.
/// Applique les techniques de deduction logique avant de recourir au backtracking.
/// </summary>
public class HumanSolver : ISudokuSolver
{
    // Historique des techniques utilisees
    public List<(string Technique, int Count)> SolveLog { get; private set; } = new();
    
    // Indique si le backtracking a ete necessaire
    public bool UsedBacktracking { get; private set; } = false;
    
    /// <summary>
    /// Resout le Sudoku en appliquant les techniques humaines puis le backtracking si necessaire.
    /// </summary>
    public SudokuGrid Solve(SudokuGrid s)
    {
        SolveLog.Clear();
        UsedBacktracking = false;
        
        // Travailler sur une copie
        var grid = (SudokuGrid)s.Clone();
        var cg = new CandidateGrid(grid);
        
        // Appliquer les techniques humaines en boucle
        ApplyHumanTechniques(cg);
        
        // Si le puzzle n'est pas resolu, utiliser le backtracking
        if (!cg.IsSolved())
        {
            UsedBacktracking = true;
            SolveLog.Add(("Backtracking (fallback)", cg.RemainingCells()));
            BacktrackingSolve(grid);
        }
        
        // Copier le resultat dans la grille originale
        for (int r = 0; r < 9; r++)
            for (int c = 0; c < 9; c++)
                s.Cells[r, c] = grid.Cells[r, c];
        
        return s;
    }
    
    /// <summary>
    /// Applique les techniques humaines de maniere iterative.
    /// A chaque progres, recommence depuis le debut (les techniques les plus simples).
    /// </summary>
    private void ApplyHumanTechniques(CandidateGrid cg)
    {
        bool progress = true;
        
        while (progress && !cg.IsSolved())
        {
            progress = false;
            
            // 1. Naked Singles
            int ns = HumanTechniques.ApplyNakedSingles(cg);
            if (ns > 0)
            {
                SolveLog.Add(("Naked Singles", ns));
                progress = true;
                continue; // Recommencer depuis le debut
            }
            
            // 2. Hidden Singles
            int hs = HumanTechniques.ApplyHiddenSingles(cg);
            if (hs > 0)
            {
                SolveLog.Add(("Hidden Singles", hs));
                progress = true;
                continue;
            }
            
            // 3. Naked Pairs
            int np = HumanTechniquesIntermediate.ApplyNakedPairs(cg);
            if (np > 0)
            {
                SolveLog.Add(("Naked Pairs (eliminations)", np));
                progress = true;
                continue;
            }
            
            // 4. Locked Candidates (Pointing)
            int lp = HumanTechniquesIntermediate.ApplyLockedCandidatesPointing(cg);
            if (lp > 0)
            {
                SolveLog.Add(("Locked Candidates - Pointing (eliminations)", lp));
                progress = true;
                continue;
            }
            
            // 5. Locked Candidates (Claiming)
            int lc = HumanTechniquesIntermediate.ApplyLockedCandidatesClaiming(cg);
            if (lc > 0)
            {
                SolveLog.Add(("Locked Candidates - Claiming (eliminations)", lc));
                progress = true;
                continue;
            }
            
            // 6. X-Wing
            int xw = HumanTechniquesAdvanced.ApplyXWing(cg);
            if (xw > 0)
            {
                SolveLog.Add(("X-Wing (eliminations)", xw));
                progress = true;
                continue;
            }
        }
    }
    
    /// <summary>
    /// Backtracking simple utilise en dernier recours.
    /// </summary>
    private bool BacktrackingSolve(SudokuGrid grid)
    {
        // Trouver la premiere cellule vide
        for (int row = 0; row < 9; row++)
        {
            for (int col = 0; col < 9; col++)
            {
                if (grid.Cells[row, col] == 0)
                {
                    foreach (int val in grid.GetAvailableNumbers(row, col))
                    {
                        grid.Cells[row, col] = val;
                        if (BacktrackingSolve(grid))
                            return true;
                        grid.Cells[row, col] = 0;
                    }
                    return false;
                }
            }
        }
        return true; // Toutes les cellules remplies
    }
    
    /// <summary>
    /// Affiche le journal de resolution.
    /// </summary>
    public string GetSolveReport()
    {
        var sb = new StringBuilder();
        sb.AppendLine("--- Journal de resolution ---");
        int step = 1;
        foreach (var (technique, count) in SolveLog)
        {
            sb.AppendLine($"  Etape {step++}: {technique} x{count}");
        }
        sb.AppendLine($"  Backtracking necessaire : {(UsedBacktracking ? "OUI" : "NON")}");
        sb.AppendLine($"  Total etapes : {SolveLog.Count}");
        return sb.ToString();
    }
}

### Interpretation : Architecture du HumanSolver

Le `HumanSolver` suit un pattern classique de **pipeline de strategies** :

```
Boucle principale
  |-> Naked Singles (basique)
  |-> Hidden Singles (basique)
  |-> Naked Pairs (intermediaire)
  |-> Locked Candidates Pointing (intermediaire)
  |-> Locked Candidates Claiming (intermediaire)
  |-> X-Wing (avance)
  |-> [Si bloque] Backtracking (fallback)
```

Le point cle est le `continue` apres chaque progres : des qu'une technique fait avancer la resolution, on recommence au debut. Cela maximise l'utilisation des techniques simples (rapides) avant de passer aux techniques complexes (couteuses).

Le journal de resolution (`SolveLog`) permet d'analyser quelles techniques ont ete necessaires pour chaque puzzle, ce qui est directement lie a la difficulte du puzzle.

### Test du solveur humain complet

Testons le `HumanSolver` sur des puzzles de chaque difficulte et observons les techniques utilisees.

In [ ]:
var humanSolver = new HumanSolver();

// Test sur puzzle facile
display("=== PUZZLE FACILE ===");
var easy = (SudokuGrid)SudokuHelper.GetSudokus(SudokuDifficulty.Easy).First().Clone();
var easySolved = humanSolver.Solve(easy);
display($"Erreurs : {easySolved.NbErrors(SudokuHelper.GetSudokus(SudokuDifficulty.Easy).First())}");
display(humanSolver.GetSolveReport());

// Test sur puzzle moyen
display("\n=== PUZZLE MOYEN ===");
var medium = (SudokuGrid)SudokuHelper.GetSudokus(SudokuDifficulty.Medium).First().Clone();
humanSolver = new HumanSolver();
var mediumSolved = humanSolver.Solve(medium);
display($"Erreurs : {mediumSolved.NbErrors(SudokuHelper.GetSudokus(SudokuDifficulty.Medium).First())}");
display(humanSolver.GetSolveReport());

// Test sur puzzle difficile
display("\n=== PUZZLE DIFFICILE ===");
var hard = (SudokuGrid)SudokuHelper.GetSudokus(SudokuDifficulty.Hard).First().Clone();
humanSolver = new HumanSolver();
var hardSolved = humanSolver.Solve(hard);
display($"Erreurs : {hardSolved.NbErrors(SudokuHelper.GetSudokus(SudokuDifficulty.Hard).First())}");
display(humanSolver.GetSolveReport());

### Interpretation : Resultats du HumanSolver

Le journal de resolution montre quelles techniques sont necessaires selon la difficulte :

| Difficulte | Techniques utilisees | Backtracking |
|------------|---------------------|-------------|
| Easy | Naked Singles + Hidden Singles | Non |
| Medium | + Naked Pairs, Locked Candidates | Possible |
| Hard | + X-Wing | Probable |

> **Observation attendue** : Les puzzles "Hard" de notre fichier de test (`Sudoku_top95.txt`) sont parmi les plus difficiles connus. Il est normal que le backtracking soit necessaire pour une partie d'entre eux, car notre solveur n'implemente pas les techniques les plus avancees (Swordfish, 3D Medusa, Unique Rectangles).

### Comparaison avec les autres solveurs

Comparons maintenant les performances du `HumanSolver` avec le `BacktrackingDotNetSolver` du notebook 1. Le solveur humain est concu pour etre **comprehensible**, pas necessairement rapide. Voyons comment il se compare en termes de temps d'execution.

In [ ]:
var solvers = new List<(string Name, ISudokuSolver Solver)>
{
    ("HumanSolver", new HumanSolver()),
    ("BacktrackingDotNetSolver", new BacktrackingDotNetSolver())
};

var results = SudokuHelper.TestSolvers(solvers, numberOfSudokus: 10, timeLimitMilliseconds: 5000);

// Affichage des resultats
display("\nResultats de performance :");
foreach (var result in results)
{
    display($"{result.SolverName} | {result.Difficulty} | {result.Time:F1} ms | {result.SolvedCount}/10 | {result.Status}");
}

SudokuHelper.DisplayResults(results);

### Interpretation : Comparaison de performance

| Solveur | Forces | Faiblesses |
|---------|--------|------------|
| HumanSolver | Explicable, tracable, pedagogique | Plus lent (techniques d'elimination) |
| Backtracking | Simple, rapide sur puzzles faciles | Force brute, non explicable |

Le `HumanSolver` sera generalement plus lent que le backtracking pur car chaque technique d'elimination a un cout. Cependant, il offre un avantage majeur : **chaque etape est explicable**. On sait exactement pourquoi une valeur a ete placee ou un candidat elimine.

> **Note** : Dans les applications reelles (generateurs de puzzles, tutoriels interactifs), la capacite a tracer le raisonnement est plus importante que la vitesse brute. C'est ce qui permet de classer les puzzles par difficulte et d'aider les joueurs a progresser.

### Analyse statistique par difficulte

Analysons plus en detail les techniques utilisees sur un ensemble de puzzles pour comprendre la distribution des techniques necessaires par niveau de difficulte.

In [ ]:
// Analyse des techniques utilisees sur un echantillon de puzzles
void AnalyzeTechniques(SudokuDifficulty difficulty, int count)
{
    var puzzles = SudokuHelper.GetSudokus(difficulty).Take(count).ToList();
    var techniqueCounts = new Dictionary<string, int>();
    int backtrackCount = 0;
    
    foreach (var puzzle in puzzles)
    {
        var solver = new HumanSolver();
        var clone = (SudokuGrid)puzzle.Clone();
        solver.Solve(clone);
        
        if (solver.UsedBacktracking) backtrackCount++;
        
        foreach (var (technique, _) in solver.SolveLog)
        {
            var baseName = technique.Split('(')[0].Trim();
            if (!techniqueCounts.ContainsKey(baseName))
                techniqueCounts[baseName] = 0;
            techniqueCounts[baseName]++;
        }
    }
    
    display($"\n=== {difficulty} ({count} puzzles) ===");
    foreach (var kvp in techniqueCounts.OrderByDescending(x => x.Value))
    {
        display($"  {kvp.Key}: {kvp.Value} utilisations ({100.0 * kvp.Value / puzzles.Count:F0}% des puzzles)");
    }
    display($"  Backtracking necessaire : {backtrackCount}/{count} puzzles ({100.0 * backtrackCount / count:F0}%)");
}

AnalyzeTechniques(SudokuDifficulty.Easy, 20);
AnalyzeTechniques(SudokuDifficulty.Medium, 10);
AnalyzeTechniques(SudokuDifficulty.Hard, 10);

### Interpretation : Distribution des techniques

Cette analyse revele la **signature de difficulte** de chaque niveau :

- **Easy** : Principalement Naked Singles et Hidden Singles. Aucun backtracking necessaire.
- **Medium** : Apparition des Naked Pairs et Locked Candidates. Le backtracking peut etre necessaire si les techniques avancees ne sont pas implementees.
- **Hard** : Les techniques avancees (X-Wing) sont mobilisees. Le taux de backtracking augmente car il faudrait des techniques expertes (Swordfish, 3D Medusa) pour eviter le fallback.

> **Conclusion** : Le nombre et la complexite des techniques necessaires constituent une mesure naturelle de la difficulte d'un puzzle. C'est d'ailleurs ainsi que les generateurs de puzzles classifient leurs grilles.

## 8. Exercices

### Exercice 1 : Suivi detaille des techniques

Modifiez le `HumanSolver` pour enregistrer non seulement la technique utilisee mais aussi la **cellule** et la **valeur** concernees. Affichez un journal de resolution detaille, cellule par cellule.

**Indice** : Ajoutez un champ `(int row, int col, int value)` aux entries du `SolveLog`.

### Exercice 2 : Implementation du Swordfish

Implementez la technique Swordfish en generalisant le X-Wing a 3 lignes et 3 colonnes.

**Indice** : Au lieu de chercher 2 lignes avec 2 colonnes identiques, cherchez 3 lignes ou un candidat est present dans au plus 3 colonnes, et ou l'union des colonnes donne exactement 3 colonnes.

```csharp
// Squelette pour le Swordfish
public static int ApplySwordfish(CandidateGrid cg)
{
    int eliminations = 0;
    for (int val = 1; val <= 9; val++)
    {
        // Pour chaque triplet de lignes (r1, r2, r3)
        // Collecter les colonnes ou val est candidat dans chaque ligne
        // Si l'union des colonnes = exactement 3 colonnes
        // Eliminer val des autres cellules de ces 3 colonnes
    }
    return eliminations;
}
```

## 9. Conclusion

### Resume des apprentissages

Ce notebook a presente une approche de resolution de Sudoku basee sur les techniques humaines de deduction logique. Nous avons implemente 6 techniques (Naked Singles, Hidden Singles, Naked Pairs, Locked Candidates Pointing/Claiming, X-Wing) et les avons combinees dans un solveur hybride.

| Technique implementee | Type | Action | Difficulte cible |
|-----------------------|------|--------|------------------|
| Naked Singles | Basique | Placement | Easy |
| Hidden Singles | Basique | Placement | Easy |
| Naked Pairs | Intermediaire | Elimination | Medium |
| Locked Candidates (Pointing) | Intermediaire | Elimination | Medium |
| Locked Candidates (Claiming) | Intermediaire | Elimination | Medium |
| X-Wing | Avance | Elimination | Hard |
| Backtracking (fallback) | Dernier recours | Exploration | Expert |

### Points cles

1. **Les techniques humaines sont hierarchiques** : on commence par les plus simples et on monte en complexite uniquement quand on est bloque
2. **Elimination vs placement** : les techniques avancees eliminent des candidats, ce qui deblocke les techniques basiques
3. **La difficulte d'un puzzle** se mesure par la technique la plus avancee necessaire pour le resoudre sans backtracking
4. **Le backtracking comme filet de securite** : un solveur humain incomplet (sans toutes les 13 techniques) peut toujours resoudre en dernier recours via le backtracking

### Perspectives

Les techniques humaines offrent un avantage majeur sur la force brute : **chaque etape est explicable**. Cette propriete est essentielle pour les applications pedagogiques (tutoriels interactifs) et pour la generation de puzzles de difficulte controlee.

> **Note** : Le projet [Sudoku.Human](https://github.com/jsboigeEpita/2024-EPITA-SCIA-PPC-Sudoku-NLP) implemente les 13 techniques dans 23 fichiers C#, offrant une couverture complete des patterns de resolution humaine.

---

<< [Sudoku-8-SimulatedAnnealing](Sudoku-8-SimulatedAnnealing.ipynb) | [Index](README.md) | [Sudoku-10-NeuralNetwork](Sudoku-10-NeuralNetwork.ipynb) >>

**Voir aussi** : 
- [Search-7-CSP-Consistency](../Search/Foundations/Search-7-CSP-Consistency.ipynb) - Propagation de contraintes
- [Sudoku-7-Norvig](Sudoku-7-Norvig.ipynb) - Propagation de Norvig